## 1. Implementation Overview

This project integrates multiple phishing-related datasets into a unified database for embedding and retrieval using a RAG-based LLM architecture. The pipeline includes:

- **Data Ingestion**: Seven datasets loaded via `pandas`, including CEAS, Enron, Ling, Nazario, Nigerian Fraud, phishing_email.csv, and SpamAssassin.
- **Preprocessing**:
  - Concatenation of `subject` and `body` into a single `text` field.
  - Filtering of emails with fewer than 20 characters to reduce noise.
- **Document Construction**: Each email is wrapped in a `Document` object with metadata indicating phishing status (`label`).
- **Embedding**: Texts are embedded using OpenAI's API and stored in a FAISS vectorstore.
- **Retrieval-Augmented Generation (RAG)**: Embedded documents are queried to support phishing classification and explanation via LLM..

## 2. Technical Architecture

- **Language Model**: OpenAI LLM (via API)
- **Embedding Model**: `text-embedding-ada-002`
- **Vectorstore**: FAISS with metadata filtering
- **Document Format**: LangChain `Document` objects with `page_content` and `label` metadata
- **Environment**: Python 3.12, LangChain, Pandas, FAISS, dotenv

## 3. Challenges Encountered

- **Restart Overhead**: Initial runs of the embedding pipeline were vulnerable to interruptions (e.g., API timeouts, memory limits), requiring full restarts that could take 30–45 minutes depending on batch size and dataset volume.
- **Checkpointing Logic**: Embedding progress was not initially tracked
- **Batch Management**: Large batches risked exceeding token limits or triggering rate caps; small batches increased total runtime and complexity.
- **Embedding Limits**: OpenAI API rate limits and token constraints required batching and retry logic.

To address these, the pipeline was refactored to include:
- Persistent checkpointing via batch index tracking.
- Modular embedding functions with resume capability.
- FAISS index saving after each batch to prevent data loss.
- Logging for each stage to support auditability and debugging.
- Skipping the two largest emails

## 4. Key Learnings

- **Modular Preprocessing** enables flexible dataset integration and rapid debugging.
- **Metadata-Driven Retrieval** improves interpretability and classification accuracy.
- **Audit-Friendly Design** (e.g., explicit label mapping, resume-safe batching) is essential for cybersecurity applications.

In [3]:
import pandas as pd

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE" 


In [7]:
df_01 = pd.read_csv("Phishing_Email_Data/CEAS_08.csv")
df_02 = pd.read_csv("Phishing_Email_Data/Enron.csv")
df_03 = pd.read_csv("Phishing_Email_Data/Ling.csv")
df_04 = pd.read_csv("Phishing_Email_Data/Nazario.csv")
df_05 = pd.read_csv("Phishing_Email_Data/Nigerian_Fraud.csv")
df_06 = pd.read_csv("Phishing_Email_Data/phishing_email.csv")
df_07 = pd.read_csv("Phishing_Email_Data/SpamAssasin.csv")


In [ ]:
df = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_07], ignore_index=True)

In [9]:
# Combine subject and body into a single text field per email
df["text"] = df["subject"].fillna("") + " " + df["body"].fillna("")
# Filter out emails that are empty or too short to be meaningful
df = df[df["text"].str.strip().str.len() > 20] 


In [10]:
df.head()

,sender,receiver,date,subject,body,label,urls,text_combined,text
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1.0,NaN,"Never agree to be a loser Buck up, your troubl..."
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1.0,NaN,Befriend Jenna Jameson \nUpgrade your sex and ...
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1.0,NaN,CNN.com Daily Top 10 >+=+=+=+=+=+=+=+=+=+=+=+=...
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1.0,NaN,Re: svn commit: r619753 - in /spamassassin/tru...
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1.0,NaN,SpecialPricesPharmMoreinfo \nWelcomeFastShippi...


In [11]:
df["label"].value_counts()

label
1    42882
0    39592
Name: count, dtype: int64

In [35]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# Specify UTF-8 encoding (most common for text files)
loader = TextLoader("security_guidelines.txt", encoding="utf-8")  
security_docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
security_chunks = splitter.split_documents(security_docs)

In [38]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os

# Create embeddings model — transforms text chunks into vectors
embedding = OpenAIEmbeddings()

# Build FAISS vectorstore from your chunked security guideline documents
vectorstore_guidelines = FAISS.from_documents(security_chunks, embedding)

# Define a separate folder path for saving security guidelines index and metadata
GUIDELINES_INDEX_PATH = "security_faiss_index"

# Make sure the directory exists
os.makedirs(GUIDELINES_INDEX_PATH, exist_ok=True)

# Save the security guidelines vectorstore locally (index + metadata)
vectorstore_guidelines.save_local(GUIDELINES_INDEX_PATH)


In [ ]:
from langchain.schema import Document

# Create a list of Document objects from the DataFrame `df`
documents = [
    Document(
        # Use the combined email text from the 'text' column as the main content
        page_content=row['text'],  
        
        # Add metadata to each document indicating whether it's phishing or not,
        # based on the 'label' column (1 means phishing, otherwise not phishing)
        metadata={"label": "Phishing" if row['label'] == 1 else "Not Phishing"}
    )
    # Loop through each row of the DataFrame using `iterrows()`
    for _, row in df.iterrows()
]


In [ ]:
import os
import json
import time
from tqdm import tqdm
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import tiktoken

# Setup
embedding_model = OpenAIEmbeddings()
tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")

MAX_TOKENS_PER_BATCH = 290_000  # keep below 300k limit for batch total tokens
MAX_TOKENS_PER_DOC = 290_000    # max tokens allowed per single document to embed
SLEEP_TIME = 1.5  # rate limit buffer
INDEX_PATH = "faiss_index"
PROGRESS_LOG = os.path.join(INDEX_PATH, "embedding_progress.json")

# Make sure the index folder exists
os.makedirs(INDEX_PATH, exist_ok=True)

# Load your documents (list of Document objects)
# documents = [ ... your docs from DataFrame as before ...]

def num_tokens(text):
    return len(tokenizer.encode(text))

# Load progress log or initialize it
if os.path.exists(PROGRESS_LOG):
    with open(PROGRESS_LOG, "r") as f:
        progress_data = json.load(f)
        start_idx = progress_data.get("last_index", 0) + 1
        print(f"Resuming from document index: {start_idx}")
else:
    start_idx = 0

vectorstore = None
current_batch = []
current_tokens = 0

# If index exists already, load it
faiss_index_file = os.path.join(INDEX_PATH, "index.faiss")
faiss_metadata_file = os.path.join(INDEX_PATH, "index.pkl")
if os.path.exists(faiss_index_file) and os.path.exists(faiss_metadata_file):
    vectorstore = FAISS.load_local(INDEX_PATH, embedding_model, allow_dangerous_deserialization=True)
    print("Loaded existing FAISS index from disk.")
else:
    print("No existing index found, starting fresh.")

for i in tqdm(range(start_idx, len(documents)), desc="Embedding batches"):
    doc = documents[i]
    doc_tokens = num_tokens(doc.page_content)

    # Skip docs that are too large to embed in one request
    if doc_tokens > MAX_TOKENS_PER_DOC:
        print(f"Skipping document {i} with {doc_tokens} tokens — too large to embed.")
        with open(PROGRESS_LOG, "w") as f:
            json.dump({"last_index": i}, f)
        continue

    # If adding this doc would exceed batch token limit, process current batch
    if current_tokens + doc_tokens > MAX_TOKENS_PER_BATCH and current_batch:
        texts = [d.page_content for d in current_batch]
        metadatas = [d.metadata for d in current_batch]

        batch_index = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)
        if vectorstore is None:
            vectorstore = batch_index
        else:
            vectorstore.merge_from(batch_index)

        # Save index and progress log after batch embedding
        vectorstore.save_local(INDEX_PATH)
        with open(PROGRESS_LOG, "w") as f:
            json.dump({"last_index": i - 1}, f)

        time.sleep(SLEEP_TIME)
        current_batch = []
        current_tokens = 0

    # Add current doc to batch
    current_batch.append(doc)
    current_tokens += doc_tokens

# Process any remaining documents after loop ends
if current_batch:
    texts = [d.page_content for d in current_batch]
    metadatas = [d.metadata for d in current_batch]

    batch_index = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)
    if vectorstore is None:
        vectorstore = batch_index
    else:
        vectorstore.merge_from(batch_index)

    vectorstore.save_local(INDEX_PATH)
    with open(PROGRESS_LOG, "w") as f:
        json.dump({"last_index": len(documents) - 1}, f)

print("Embedding complete and saved.")


Resuming from document index: 71769
Loaded existing FAISS index from disk.


Embedding batches:   4%|▍         | 430/10705 [00:01<00:24, 423.14it/s]

Skipping document 71769 with 2955173 tokens — too large to embed.


Embedding batches:   6%|▋         | 673/10705 [00:01<00:19, 515.39it/s]

Skipping document 72302 with 380571 tokens — too large to embed.


Embedding batches: 100%|██████████| 10705/10705 [12:16<00:00, 14.53it/s]


Embedding complete and saved.


In [ ]:
# Custom RetrievalQA subclass that changes how retrieved documents are combined
class CustomRetrievalQA(RetrievalQA):
    def combine_documents(self, docs, question, **kwargs):
        # Instead of just concatenating document text,
        # we first call `format_docs_with_labels(docs)`,
        # which injects the "Label: ..." metadata into the text for the LLM to see.
        similar_emails_text = format_docs_with_labels(docs)
        
        # Then we pass that labeled text into a custom prompt template,
        # so the LLM receives both the content AND the classification label
        # for each retrieved example email.
        return custom_prompt.format(similar_emails=similar_emails_text, query=question)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from pydantic import BaseModel, Field
from langchain.schema.retriever import BaseRetriever

# Load GPT-3.5 Turbo with deterministic output (temperature=0)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Create retrievers that pull the 3 most semantically similar chunks per query
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever_guidelines = vectorstore_guidelines.as_retriever(search_type="similarity", search_kwargs={"k": 3})


# This helper takes each retrieved Document and
# prepends its stored phishing label from metadata before the actual content.
# This is the **critical step** that ensures the LLM sees both the label and the email text.
def format_docs_with_labels(docs):
    parts = []
    for doc in docs:
        # Grab label from metadata (added when docs were stored in the vector DB)
        label = doc.metadata.get("label", "Unknown")
        # Create a readable chunk: "[Label: X]\n<email text>"
        parts.append(f"[Label: {label}]\n{doc.page_content}")
    # Separate each doc with a visual divider
    return "\n\n---\n\n".join(parts)


# Wrapper retriever that doesn't change the retrieval logic
# but makes it easy to inject our label-formatting step later.
class LabelInjectingRetriever(BaseRetriever, BaseModel):
    base_retriever: BaseRetriever = Field(...)

    def get_relevant_documents(self, query):
        # Just fetch the raw docs (labels already live in metadata)
        return self.base_retriever.get_relevant_documents(query)

    async def aget_relevant_documents(self, query):
        return self.get_relevant_documents(query)

    
# Wrap the original retrievers with our label-aware wrapper
wrapped_retriever = LabelInjectingRetriever(base_retriever=retriever)
wrapped_retriever_guidelines = LabelInjectingRetriever(base_retriever=retriever_guidelines)

# Standard RetrievalQA chain, but since we're using the wrapped retriever,
# any docs retrieved still have their label metadata available for formatting.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=wrapped_retriever,
    return_source_documents=True
)

qa_chain_guidelines = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_guidelines,
    return_source_documents=True
)


C:\Users\T\AppData\Local\Temp\ipykernel_4484\83177682.py:27: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class LabelInjectingRetriever(BaseRetriever, BaseModel):
C:\Users\T\AppData\Local\Temp\ipykernel_4484\83177682.py:27: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class LabelInjectingRetriever(BaseRetriever, BaseModel):


In [ ]:
# Example email to classify
email = """
Subject: Urgent Password Reset

Dear User,

Your account has been flagged. Please reset your password immediately using the link below or your account will be suspended.

[Reset Now]

Regards,
Admin
"""


def classify_email(email_text):
    # Step 1: Retrieve similar example emails from the vectorstore
    similar_docs = wrapped_retriever.get_relevant_documents(email_text)
    
    # Step 2: Inject each doc's label into the visible text for the LLM
    labeled_similar_emails = format_docs_with_labels(similar_docs)
    
    # Step 3: Construct a prompt that shows the LLM:
    #   - The labeled examples (from Step 2)
    #   - The new email that needs classifying
    prompt_text = (
        "You are a cybersecurity expert. Here are some similar emails with their phishing labels:\n\n"
        f"{labeled_similar_emails}\n\n"
        "Please carefully analyze these examples and their labels.\n"
        "Using that information, classify the following new email as 'Phishing' or 'Not Phishing'.\n"
        "In your explanation, explicitly reference how the examples influenced your decision.\n\n"
        f"New email:\n{email_text}"
    )
    
    # Step 4: Ask the LLM to make a classification and explain
    response = llm.predict(prompt_text)
    
    # Return classification + the docs that were shown to the LLM
    return response.strip(), similar_docs

classification, similar_docs = classify_email(email)

# Now we can also get recommendations from the two QA chains
advice_from_emails = qa_chain.invoke(
    f"What are the recommended actions according to cybersecurity best practices if I receive an email like this, , if it's not a phishing email, don't give any advice:\n\n{email}"
)

advice_from_guidelines = qa_chain_guidelines.invoke(
    f"What are the recommended actions according to cybersecurity best practices if I receive an email like this, , if it's not a phishing email, don't give any advice:\n\n{email}"
)

print("Classification:", classification)
print("\nAdvice from similar emails:\n", advice_from_emails['result'])
print("\nAdvice from security guidelines:\n", advice_from_guidelines['result'])

Classification: Based on the examples provided, the new email should be classified as 'Phishing'. The common characteristics among the labeled phishing emails include urgent language, threats of account suspension, requests to click on suspicious links, and claims of unusual activities on the account. 

The new email exhibits similar characteristics such as urgency in the subject line, a warning about the account being flagged, and a request to reset the password immediately using a link provided in the email. These are all red flags commonly associated with phishing attempts. 

Therefore, based on the patterns observed in the labeled phishing emails, it is likely that the new email is also a phishing attempt.

Advice from similar emails:
 The email you received seems to be a phishing attempt. Here are some recommended actions according to cybersecurity best practices:

1. **Do not click on any links**: Avoid clicking on any links or downloading any attachments in the email. These link

In [40]:
print("Similar Emails Retrieved and Their Labels:\n")
print(format_docs_with_labels(similar_docs))

Similar Emails Retrieved and Their Labels:

[Label: Phishing]
[Label: Phishing]
[Label: Phishing]
[Label: Phishing]
[Label: Phishing]
[Label: Phishing]
Rv-Staff-Email Access Dear Staff


Your e-mailbox password will soon expire. to validate your e-mail or you will be temporary block Please confirm the link below to Update your e-mail


Click the link https://au-mai.webnode.com/


Thank you,

IT Support

CONFIDENTIALITY NOTE:  The information contained in this transmission may contain privileged and confidential information, including patient information protected by federal and state privacy laws. It is intended only for the use of the person(s) named above. If you are not the intended recipient, you are hereby notified that any review, dissemination, distribution, or duplication of this communication is strictly prohibited. Please contact the sender by reply email and destroy all copies of the original message. 




---

[Label: Phishing]
[Label: Phishing]
[Label: Phishing]
[Label: Ph